## Урок 5. Selenium в Python

In [1]:
!pip install selenium

In [2]:
!pip install pymongo

In [3]:
#импорт библиотек
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException
from datetime import datetime, timedelta
from pymongo import MongoClient
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

import unittest
import time
import locale
import json

#### 1) Написать программу, которая собирает входящие письма из своего или тестового почтового ящика и сложить данные о письмах в базу данных
- от кого,
- дата отправки,
- тема письма,
- текст письма полный.

Логин тестового ящика: study.ai_172@mail.ru

Пароль тестового ящика: NextPassword172

In [4]:
#опции
chrome_options = Options()
chrome_options.add_argument('start-maximized')

#путь до драйвера
service = Service(executable_path='../chromedriver')

#драйвер
driver = webdriver.Chrome(service=service,options=chrome_options)
driver.get('https://m.mail.ru/login') #мобильная версия
#driver.get('https://mail.ru')

print(driver.title)
assert "Вход — Почта Mail.Ru" in driver.title

#БД
client = MongoClient('localhost', 27017)
db = client['mail_inbox_db']
collection = db.letters

#время на загрузку
authorization = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.NAME, "Login")))

#авторизация
authorization.send_keys('study.ai_172@mail.ru')
authorization = driver.find_element_by_name('Password')
authorization.send_keys('NextPassword172')
authorization.send_keys(Keys.RETURN)

#переменные
def get_message_info(lnk):
    info = {}
    driver.get(lnk)
    time.sleep(2)
    theme = driver.find_element_by_xpath("//td[@class='readmsg__theme-box__line']/span")
    info["theme"] = theme.text
    msg_from = driver.find_element_by_xpath("//div[@class='readmsg__text-container']/div[1]/a")
    when = driver.find_element_by_xpath("//div[@class='readmsg__text-container']/div[2]/div/span")
    info["from"] = msg_from.text
    info["date"] = when.text
    body = driver.find_element_by_id("readmsg__body")
    info["body"] = body.text
    driver.back()
    return info

#сбор информации
msg_list = []
while True:
    time.sleep(2)
    l_links = WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.XPATH,"//a[@class='messageline__link']")))
    print(len(l_links))
    link_list = []
    for link in l_links:
        link_list.append(link.get_attribute("href"))

    for href in link_list:
        info = get_message_info(href)
        msg_list.append(info)
    try:
        a_next = WebDriverWait(driver, 3).until(EC.presence_of_element_located((By.XPATH, "//a[@title='Следующая страница']")))
        driver.get(a_next.get_attribute("href"))
    except:
        break

#формирование файла
try:
    with open('mail.json', 'w', encoding='utf-8') as file:
        json.dump(msg_list, file, indent=2, ensure_ascii=False)
    print(f'Информация о письмах сохранена в файл json')
except:
    print(f'Ой-ёй...файл...')
    
#формирование БД    
client = MongoClient("localhost",27017)
db = client["mailru"]
db_msgs = db.messages
try:
    db_msgs.insert_many(msg_list)
    print(f'Сформирована БД по найденным письмам')
except:
    print("Что-то пошло не так с ДБ")

client.close()
driver.quit()

Вход — Почта Mail.Ru


/Users/alenakukhta/opt/anaconda3/lib/python3.8/site-packages/selenium/webdriver/remote/webdriver.py:572: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")


25


/Users/alenakukhta/opt/anaconda3/lib/python3.8/site-packages/selenium/webdriver/remote/webdriver.py:452: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")
/Users/alenakukhta/opt/anaconda3/lib/python3.8/site-packages/selenium/webdriver/remote/webdriver.py:412: UserWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  warnings.warn("find_element_by_* commands are deprecated. Please use find_element() instead")


3
Информация о письмах сохранена в файл json
Сформирована БД по найденным письмам


#### 2) Написать программу, которая собирает «Хиты продаж» с сайта техники mvideo и складывает данные в БД. 
Магазины можно выбрать свои. 
Главный критерий выбора: динамически загружаемые товары

In [5]:
#опции
chrome_options = Options()
chrome_options.add_argument('start-maximized')

#путь до драйвера
service = Service(executable_path='../chromedriver')

#драйвер
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.get('https://www.mvideo.ru/')

#переменные
time.sleep(5)
item_xpath = "//div[contains(text(), 'Хиты продаж')]/../../.."
next_button = "//a[contains(@href, '#')][2]"
product_info = "//div/ul/li//h4/a"
pause = WebDriverWait(driver, 10)

#сбор информации
try:
    next = pause.until(EC.presence_of_element_located((By.XPATH, item_xpath + next_button)))
    for x in range(5):
        next.click()
        time.sleep(2)
    items = driver.find_elements_by_xpath(item_xpath + product_info)
    hits = []
    for item in items:
        product_info = json.loads(item.get_attribute('data-product-info'))
        hits.append(product_info)
finally:
    print(f'Информация собрана по {len(hits)} найденным товаров блока Хиты продаж')
    driver.close()

#формировние файла
try:
    with open('mvideo.json', 'w', encoding='utf-8') as file:
        json.dump(hits, file, indent=2, ensure_ascii=False)
    print(f'Информация по найденным товарам сохранена в файл json')
except:
    print(f'Ой-ёй...файл...')

#формирование БД
try:
    client = MongoClient('localhost', 27017)
    db = client['Downloads']
    collection = db.mvideo
    collection.delete_many({})
    for hit in hits:
        try:
            collection.insert_one(hit)
        except:
            print(f'то-то пошло не так: {hit}')
    print(f'Сформирована БД по найденным товарам')
except:
    print(f'Ух...БД...')

/Users/alenakukhta/opt/anaconda3/lib/python3.8/site-packages/selenium/webdriver/remote/webdriver.py:471: UserWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  warnings.warn("find_elements_by_* commands are deprecated. Please use find_elements() instead")


Информация собрана по 16 найденным товаров блока Хиты продаж
Информация по найденным товарам сохранена в файл json
Сформирована БД по найденным товарам
